In [ ]:
import requests as req
from lxml import etree
import time

# 伪装请求头
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Safari/537.36 Edg/141.0.0.0',
    'Cookie': 'pvid=1758593593075; __uv_=5899880875; WOlTvIlgRpuvid_=1013; WOlTvIlgRptime_=1758593593075; SpMLdaPxuv=m2683702925; Hm_lvt_b44eee64e7c90dde50ff4bbad90eb529=1758593593; Hm_lpvt_b44eee64e7c90dde50ff4bbad90eb529=1758593593; HMACCOUNT=87C42F37E450CE4A; Hm_lvt_5a9573957327e40b58294447cd1d8ad2=1758593593; Hm_lpvt_5a9573957327e40b58294447cd1d8ad2=1758593593'
}

for page in range(1, 100):  # 循环爬取1-99页
    url = f'https://vip.1905.com/list/o6p{page}.shtml'
    
    try:
        # 发送请求并设置编码
        response = req.get(url, headers=headers)
        response.encoding = 'utf-8'
        
        if response.status_code != 200:
            print(f'⚠️ 页面{page}请求失败 [{response.status_code}]')
            continue
            
        print(f'\n{"="*50}\n✅ 成功获取第{page}页 (长度:{len(response.text)}字符)\n{"="*50}')
        
        # 解析HTML结构
        tree = etree.HTML(response.text)
        movie_items = tree.xpath('/html/body/div[2]/div/div[2]/div/ul/li')  # 更健壮的定位
        
        if not movie_items:
            print(f"❌ 第{page}页未找到电影数据，可能页面结构已变更")
            continue
            
        # 解析每部电影数据
        for idx, item in enumerate(movie_items, 1):
            # 使用相对路径提取数据 /html/body/div[2]/div/div[2]/div/ul/li[1]/a/img
            #                    /html/body/div[2]/div/div[2]/div/ul/li[2]/a/img
            name = item.xpath('.//h3/a/span[1]//text()')
            intro = item.xpath('.//h3/a/span[2]//text()')
            date = item.xpath('.//h3/a/span[3]//text()') 
            actors = item.xpath('.//h3/a/span[5]//text()' )
            bg = item.xpath('.//h3/a/span[6]//text()')
            rating = item.xpath('.//a/span[@class="score"]/text()')
            cover = item.xpath('.//a/img/@src')
            
            # 数据处理
            movie_data = {
                "name": name[0].strip() if name else "暂无",
                "intro": intro[0].strip() if intro else "暂无",
                "date": date[0].strip() if date else "未公布",
                "actors": actors[0].replace("主演：", "") if actors else "暂无",
                "bg": bg[0].strip() if bg else "暂无背景介绍",
                "rating": rating[0].strip() if rating else "暂无评分",
                "cover": f"https:{cover[0]}" if cover else "无封面"
            }
            
            # 格式化输出
            print(f"🎬 第{idx}部: <<{movie_data['name']}>>")
            print(f"   📖 介绍: {movie_data['intro']}")
            print(f"   ⭐ 评分: {movie_data['rating']}")
            print(f"   📅 上映: {movie_data['date']}")
            print(f"   🎭 主演: {movie_data['actors']}")
            print(f"   📖 背景: {movie_data['bg']}")
            print(f"   🖼️ 封面: {movie_data['cover']}\n")
        
        # 避免频繁请求
        # time.sleep(1.0) # 设置间隔时间,单位:s(秒)
        
    except Exception as e:
        print(f'❌ 第{page}页发生异常: {str(e)}')
